<a href="https://colab.research.google.com/github/peterbabulik/QuantumAI/blob/main/133_QUBIT_EXPERIMENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qiskit qiskit-ibm-runtime -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.6/378.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 6.7 MB/s eta 0:00:00


In [3]:
import numpy as np
import time
from scipy.optimize import curve_fit
from qiskit.circuit.library import EfficientSU2, ZZFeatureMap
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator, EstimatorOptions

# --- CONFIGURATION ---
API_KEY = "Your api key here"
BACKEND_NAME = "ibm_torino"
NUM_QUBITS = 133  # Full Heron Scale
SHOTS = 8192
NOISE_FACTORS = [1, 3, 5]

def tanh_decay_model(x, a, b, c):
    return a + b * np.tanh(c * x)

def run_frontier_experiment_fixed():
    print(f"--- 🚀 LAUNCHING 133-QUBIT FRONTIER EXPERIMENT (FIXED) ---")

    service = QiskitRuntimeService(channel="ibm_quantum_platform", token=API_KEY)
    backend = service.backend(BACKEND_NAME)

    # Check device size
    n_phys = backend.configuration().n_qubits
    print(f"Physical Qubits on {BACKEND_NAME}: {n_phys}")

    # --- 1. BUILD CIRCUIT ---
    print(f"Building {NUM_QUBITS}-Qubit Circuit...")
    feature_map = ZZFeatureMap(feature_dimension=NUM_QUBITS, reps=1, entanglement='linear')
    ansatz = EfficientSU2(num_qubits=NUM_QUBITS, reps=1, entanglement='linear')
    circuit = feature_map.compose(ansatz)

    # --- 2. BIND PARAMETERS LOCALLY (THE FIX) ---
    # We bind parameters here to avoid "parameter table overflow" during job serialization
    np.random.seed(1337)
    # Total params = FeatureMap inputs + Ansatz weights
    total_params = circuit.num_parameters
    random_values = np.random.uniform(0, 2*np.pi, total_params)

    print(f"Binding {total_params} parameters locally...")
    bound_circuit = circuit.assign_parameters(random_values)

    # --- 3. TRANSPILE ---
    print("Transpiling bound circuit for Heron Architecture...")
    pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
    isa_circuit = pm.run(bound_circuit)

    # Observable (Z on qubit 0)
    observable = SparsePauliOp(["I" * (NUM_QUBITS - 1) + "Z"])
    isa_observable = observable.apply_layout(isa_circuit.layout)

    # --- 4. RUN WITH ZNE ---
    print(f"Submitting Job to {BACKEND_NAME}...")

    options = EstimatorOptions()
    options.default_shots = SHOTS
    options.resilience_level = 2
    options.resilience.zne_mitigation = True
    options.resilience.zne.noise_factors = NOISE_FACTORS

    estimator = Estimator(mode=backend, options=options)

    # Note: We pass [] for params because they are already bound!
    job = estimator.run([(isa_circuit, [isa_observable], [[]])])
    print(f"Job ID: {job.job_id()}")

    print("Waiting for results (133 qubits takes time)...")
    result = job.result()

    # --- 5. ANALYSIS ---
    raw_evs = result[0].data.evs_noise_factors[0].flatten()

    print("\n" + "="*50)
    print(f"   133-QUBIT FRONTIER RESULTS")
    print("="*50)
    for nf, ev in zip(NOISE_FACTORS, raw_evs):
        print(f"Noise Factor {nf}x: {ev:.5f}")

    # Fit Linear vs Tanh
    print("-" * 50)
    slope, intercept = np.polyfit(NOISE_FACTORS, raw_evs, 1)
    zne_linear = intercept

    try:
        # Constrain Tanh to be realistic
        p0_guess = [zne_linear, -0.1, 0.5]
        popt, _ = curve_fit(tanh_decay_model, NOISE_FACTORS, raw_evs, p0=p0_guess, maxfev=5000)
        zne_tanh = popt[0]

        print(f"Linear ZNE Projection: {zne_linear:.5f}")
        print(f"Tanh ZNE Projection:   {zne_tanh:.5f}")

        # Calculate impact
        shift = abs(zne_tanh - zne_linear)
        print(f"Correction Magnitude:  {shift:.5f}")

    except Exception:
        print("Tanh fit unstable on sparse data (Linear fallback used).")

    print("="*50)
    print("SUCCESS: 133-Qubit benchmark complete.")

if __name__ == "__main__":
    try:
        run_frontier_experiment_fixed()
    except Exception as e:
        print(f"Error: {e}")

qiskit_runtime_service._discover_account:WARNING:2026-02-05 21:49:13,823: Loading account with the given token. A saved account will not be used.


--- 🚀 LAUNCHING 133-QUBIT FRONTIER EXPERIMENT (FIXED) ---


qiskit_runtime_service.__init__:WARNING:2026-02-05 21:49:16,338: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2026-02-05 21:49:16,339: Using instance: open-instance, plan: open


Physical Qubits on ibm_torino: 133
Building 133-Qubit Circuit...
Binding 665 parameters locally...
Transpiling bound circuit for Heron Architecture...


/tmp/ipython-input-1257405492.py:31: DeprecationWarning: The class ``qiskit.circuit.library.data_preparation._zz_feature_map.ZZFeatureMap`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the zz_feature_map function as a replacement. Note that this will no longer return a BlueprintCircuit, but just a plain QuantumCircuit.
  feature_map = ZZFeatureMap(feature_dimension=NUM_QUBITS, reps=1, entanglement='linear')
/tmp/ipython-input-1257405492.py:32: DeprecationWarning: The class ``qiskit.circuit.library.n_local.efficient_su2.EfficientSU2`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the function qiskit.circuit.library.efficient_su2 instead.
  ansatz = EfficientSU2(num_qubits=NUM_QUBITS, reps=1, entanglement='linear')


Submitting Job to ibm_torino...
Job ID: d62h12qo8gvs73f275lg
Waiting for results (133 qubits takes time)...

   133-QUBIT FRONTIER RESULTS
Noise Factor 1x: -0.04908
Noise Factor 3x: 0.00531
Noise Factor 5x: 0.02024
--------------------------------------------------
Linear ZNE Projection: -0.05983
Tanh ZNE Projection:   -0.09564
Correction Magnitude:  0.03581
SUCCESS: 133-Qubit benchmark complete.


/tmp/ipython-input-1257405492.py:89: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(tanh_decay_model, NOISE_FACTORS, raw_evs, p0=p0_guess, maxfev=5000)
